## Intro
Take Election tweets between AUG and OCT Look at


*   Common words
  * per like
  * trump or biden
      * sentiment analysis
* time of day?
  * need to convert

How was this scraped?



In [1]:
import pandas as pd

df_orig = pd.read_csv('/content/us_election-subset20like.csv')
df_orig

,Created At,Hashtags,Id,Language,Link,Name,Reply To,Tweet,User Id,Username,F1,Likes Count,Replies Count,Retweets Count
0,2020-10-28 19:13:51 IST,['uselection'],1321447588688584704,en,https://twitter.com/mariawirth1/status/1321447...,Maria Wirth,"{'user_id': None, 'username': None}","In 2016, almost all Germans were against Trump...",1358917686,mariawirth1,118,119,4,20
1,2020-10-28 19:09:31 IST,"['covid', 'nagornokarabakh']",1321446498794569728,en,https://twitter.com/AliTahmizian/status/132144...,Alison Meuse,"{'user_id': None, 'username': None}",Azerbaijan's offensive may have been timed to ...,394593711,alitahmizian,141,25,3,13
2,2020-10-28 19:06:28 IST,[],1321445733904617472,en,https://twitter.com/rcolvile/status/1321445733...,Robert Colvile,"{'user_id': None, 'username': None}",US election veterans - what is the best strate...,18331985,rcolvile,157,20,34,1
3,2020-10-28 18:47:09 IST,['uselection'],1321440873113112576,en,https://twitter.com/SkyNews/status/13214408731...,SkyNews,"{'user_id': None, 'username': None}",Eyewitness: Voters with doubts about both cand...,7587032,skynews,243,20,19,3
4,2020-10-28 18:38:01 IST,[],1321438571585245184,en,https://twitter.com/VABVOX/status/132143857158...,Victoria Brownworth #AntiFascistVoter,"{'user_id': None, 'username': None}",Make it all blue. Build your own US election...,138168339,vabvox,299,24,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5045,2020-10-24 00:51:08 IST,[],1319720532607143936,en,https://twitter.com/talkRADIO/status/131972053...,talkRADIO,"{'user_id': None, 'username': None}","Following last night's presidential debate, wh...",3380282686,talkradio,20424,137,60,155
5046,2020-10-24 00:46:06 IST,[],1319719262580330496,en,https://twitter.com/thehill/status/13197192625...,The Hill,"{'user_id': None, 'username': None}",NASA astronaut Kate Rubins votes from space in...,1917731,thehill,20435,623,26,82
5047,2020-10-24 00:42:35 IST,[],1319718380841144320,en,https://twitter.com/akhilkom/status/1319718380...,Akhil,"{'user_id': '18690738', 'username': '_bikerchi...",@_bikerchick @dbarrett @businessinsider I live...,32863133,akhilkom,20444,20,0,0
5048,2020-10-24 00:40:17 IST,[],1319717802182348800,en,https://twitter.com/gathara/status/13197178021...,gathara,"{'user_id': None, 'username': None}","There's nothing normal about being forced to ""...",15659814,gathara,20455,82,8,35


In [14]:
df_orig['Hashtags'].value_counts().head(15)

[]                             4035
['uselection']                   81
['untrendoctober']               20
['uselection2020']               17
['election2020']                 14
['covid19']                      12
['justvote']                     12
['vote']                         11
['breaking']                     10
['newsnight']                     9
['debates2020']                   9
['gravitas']                      8
['uselections2020']               7
['coronavirus']                   7
['uselection', 'kayburley']       7
Name: Hashtags, dtype: int64

In [15]:
import spacy
from spacy.tokenizer import Tokenizer

nlp = spacy.load("en_core_web_lg")
tokenizer = Tokenizer(nlp.vocab)

OSError: ignored